In [1]:
# declare the imports for building our neural network and displaying our data 
import PIL
import os
import h5py
import sys
import random
import csv
import cv2
import fnmatch
import tensorflow as tf
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import kerastuner as kt
from shutil import copyfile
from sklearn.model_selection import train_test_split
from numpy import asarray
from numpy import savetxt
from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, Activation, BatchNormalization
from keras.utils import np_utils
from matplotlib import image
from os import listdir
from sklearn.metrics import classification_report
from keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam



# set the input and outpout Dirs
data_dir = 'C:\\Users\davie\Desktop\Final_Project\FYP\HCurve'
Output = 'C:\\Users\davie\Desktop\Final_Project\FYP\Processed_Data\\'

In [2]:
# config settings 
num_epochs = 15
bsize = 50
EARLY_STOPPING_PATIENCE = 5
input_shape = (32, 32, 3)
n_classes = 23
class_names = ['ABDMJ', 'Agen', 'Allaple', 'Autoit', 'Berbew', 'Bitman', 'Dinwod', 'Dorkbot', 
               'Dridex', 'Emotet', 'Fsysna', 'Hematite', 'InstallMonster', 'Oberal', 'Picsys', 
               'Salgorea', 'Scar', 'Sfone', 'Shifu', 'Socks', 'Sytro', 'Vilsel', 'Vobfus']

In [3]:
def save_plot(H):
    # plot the training loss and accuracy
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(H.history["loss"], label="train_loss")
    plt.plot(H.history["val_loss"], label="val_loss")
    plt.plot(H.history["accuracy"], label="train_acc")
    plt.plot(H.history["val_accuracy"], label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend()
    plt.savefig(Output)

In [4]:
# load and examine the shape of the data 
X_train = np.load(Output+'xtrain32x32.npy')
Y_train = np.load(Output+'ytrain32x32.npy')
X_test = np.load(Output+'xtest32x32.npy')
Y_test = np.load(Output+'ytest32x32.npy')
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(9818, 32, 32, 3)
(3273, 32, 32, 3)
(9818,)
(3273,)


In [5]:
# one-hot encoding using keras
print("Shape before one-hot encoding: ", Y_train.shape)
Y_train = np_utils.to_categorical(Y_train, n_classes)
Y_test = np_utils.to_categorical(Y_test, n_classes)
print("Shape after one-hot encoding: ", Y_train.shape)

Shape before one-hot encoding:  (9818,)
Shape after one-hot encoding:  (9818, 23)


In [6]:
# initialize an early stopping callback to prevent the model from
# overfitting/spending too much time training with minimal gains
es = EarlyStopping(monitor="val_loss",patience=EARLY_STOPPING_PATIENCE,restore_best_weights=True)

In [7]:
def build_model(hp):
    # building a linear stack of layers with the sequential model
    model = Sequential()
    # convolutional layer
    model.add(Conv2D(50, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', input_shape=(32, 32, 3)))
    # convolutional layer
    model.add(Conv2D(75, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    # convolutional layer
    model.add(Conv2D(125, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    # convolutional layer
    model.add(Conv2D(150, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    # convolutional layer
    model.add(Conv2D(175, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    # flatten output
    model.add(Flatten())
    # hidden layer
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(250, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(125, activation='relu'))
    model.add(Dropout(0.2))
    # output layer
    model.add(Dense(23, activation='softmax'))

    # initialize the learning rate choices and optimizer
    lr = hp.Choice("learning_rate",values=[1e-1, 1e-2, 1e-3])
    opt = Adam(learning_rate=lr)
    # compile the model
    model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
    # return the model
    return model

In [9]:
print("[INFO] instantiating a hyperband tuner object...")
tuner = kt.Hyperband(build_model,objective="val_accuracy",max_epochs=num_epochs,factor=3,seed=42,directory=Output)

[INFO] instantiating a hyperband tuner object...


Traceback (most recent call last):
  File "C:\Anaconda\lib\site-packages\kerastuner\engine\hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-7-e9c9f33a0c9b>", line 38, in build_model
    model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
  File "C:\Users\davie\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py", line 548, in compile
    self.optimizer = self._get_optimizer(optimizer)
  File "C:\Users\davie\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py", line 586, in _get_optimizer
    return tf.nest.map_structure(_get_single_optimizer, optimizer)
  File "C:\Users\davie\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\util\nest.py", line 867, in map_structure
    structure[0], [func(*x) for x in entries],
  File "C:\Users\davie\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\util\nest.py", line 867, in <listcomp>
    structure[0], [fu

Invalid model 0/5


Traceback (most recent call last):
  File "C:\Anaconda\lib\site-packages\kerastuner\engine\hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-7-e9c9f33a0c9b>", line 38, in build_model
    model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
  File "C:\Users\davie\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py", line 548, in compile
    self.optimizer = self._get_optimizer(optimizer)
  File "C:\Users\davie\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py", line 586, in _get_optimizer
    return tf.nest.map_structure(_get_single_optimizer, optimizer)
  File "C:\Users\davie\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\util\nest.py", line 867, in map_structure
    structure[0], [func(*x) for x in entries],
  File "C:\Users\davie\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\util\nest.py", line 867, in <listcomp>
    structure[0], [fu

Invalid model 1/5


...

Invalid model 3/5


Traceback (most recent call last):
  File "C:\Anaconda\lib\site-packages\kerastuner\engine\hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-7-e9c9f33a0c9b>", line 38, in build_model
    model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
  File "C:\Users\davie\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py", line 548, in compile
    self.optimizer = self._get_optimizer(optimizer)
  File "C:\Users\davie\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py", line 586, in _get_optimizer
    return tf.nest.map_structure(_get_single_optimizer, optimizer)
  File "C:\Users\davie\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\util\nest.py", line 867, in map_structure
    structure[0], [func(*x) for x in entries],
  File "C:\Users\davie\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\util\nest.py", line 867, in <listcomp>
    structure[0], [fu

Invalid model 4/5
Invalid model 5/5


Traceback (most recent call last):
  File "C:\Anaconda\lib\site-packages\kerastuner\engine\hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-7-e9c9f33a0c9b>", line 38, in build_model
    model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
  File "C:\Users\davie\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py", line 548, in compile
    self.optimizer = self._get_optimizer(optimizer)
  File "C:\Users\davie\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py", line 586, in _get_optimizer
    return tf.nest.map_structure(_get_single_optimizer, optimizer)
  File "C:\Users\davie\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\util\nest.py", line 867, in map_structure
    structure[0], [func(*x) for x in entries],
  File "C:\Users\davie\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\util\nest.py", line 867, in <listcomp>
    structure[0], [fu

RuntimeError: Too many failed attempts to build model.

In [ ]:
# perform the hyperparameter search
print("[INFO] performing hyperparameter search...")
tuner.search(x=X_train, y=Y_train, validation_data=(X_test, Y_test), batch_size=bsize, callbacks=[es], epochs=num_epochs)
# grab the best hyperparameters
bestHP = tuner.get_best_hyperparameters(num_trials=1)[0]
print("[INFO] optimal number of filters in conv_1 layer: {}".format(bestHP.get("conv_1")))
print("[INFO] optimal number of filters in conv_2 layer: {}".format(bestHP.get("conv_2")))
print("[INFO] optimal number of units in dense layer: {}".format(bestHP.get("dense_units")))
print("[INFO] optimal learning rate: {:.4f}".format(bestHP.get("learning_rate")))

In [ ]:
# build the best model and train it
print("[INFO] training the best model...")
model = tuner.hypermodel.build(bestHP)
H = model.fit(x=trainX, y=trainY, validation_data=(testX, testY), batch_size=bsize,epochs=num_epochs, callbacks=[es], verbose=1)
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1), target_names=class_names))


In [ ]:
# building the input vector from the 32x32 pixels
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# normalizing the data to help with the training
X_train /= 255
X_test /= 255

In [ ]:
# initialize an early stopping callback to prevent the model from
# overfitting/spending too much time training with minimal gains
es = EarlyStopping(monitor="val_loss",patience=EARLY_STOPPING_PATIENCE,restore_best_weights=True)

In [ ]:
# training the model for 15 epochs with a batch size of 50 as there are memory constraints and tweeking the params
history = model.fit(X_train, Y_train, batch_size=100, epochs=num_epochs, validation_data=(X_test, Y_test))

model.summary()

In [ ]:
# evaluate the model and print the score to screen 
trainingscores = model.evaluate(X_train, Y_train, verbose=0)
testingscores = model.evaluate(X_test, Y_test, verbose=0)
print("[INFO] evaluating network")
predictions = model.predict(X_test, batch_size=100)
print("training scores are below \n")
print("%s: %.2f%%" % (model.metrics_names[1], trainingscores[1]*100))
print("testing scores are below \n")
print("%s: %.2f%%" % (model.metrics_names[1], testingscores[1]*100))

In [ ]:
print(classification_report(Y_test.argmax(axis=1), predictions.argmax(axis=1), target_names=class_names))


In [ ]:
# Declare vars for the plotting of figures 
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))

# overall model scoreing 
plt.plot(val_loss, 'r', loss, 'b', val_accuracy, 'g', accuracy, 'y')
label = 'R = val_loss', 'B = loss', 'G = val_accuracy', 'Y = accuarcy'
plt.title('Model Overview')
plt.legend(label)
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.show()

# Training output 
plt.plot(epochs, accuracy, label='Training accuracy')
plt.plot(epochs, val_accuracy, label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.show()

# testing output 
plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.show()

In [ ]:
model.save(Output + "modelReFIT6.h5")
print("Saved model to disk")